## The Zoning Problem 
The aim of this notebook is to generate the optimal zoning by formulating a linear assignment problem and using google
OR-Toolsfor solving the optimization problem.
The agent behaviours are taken from the previous study. The occupation and env lattices are random for the test problem .

### The Test Problem 
There are five agents :[Blue,Green,Yellow,Red,Violet]
The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1]
Total number of voxels in a lattice = x 
Value lattice for the agents are =[Bv],[Gv],[Yv],[Rv],[Vv]

####  Aim 
: To find the best combination of Zones to achieve the maximum occupancy value
#### Steps

1. Generate the lattices , Agent behaviours, agents, 
2. Find the possible origin locations for the agents
3. Simulate occupancy behaviour and retrieve the cost of occupancy for each agent at each position
4. Generate the Cost matrix
5. Use the MIP Solver to optimise the combination to get the permutation matrix 


## Initilization

In [1]:
import os
import itertools
import sys
from math import factorial as fac
sys.path.append("D:/TU_Delft/Msc_Building_Technology/Semester_3/Graduation/Aditya_Graduation_Project_BT/06_Libraries")
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import networkx as nx
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
import networkx as nx
from itertools import combinations
import pickle

## Base Lattices

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('voxelized_envelope_6m_voxel_size.csv')
avail_lattice_base = tg.lattice_from_csv(lattice_path)
avail_lattice = avail_lattice_base*1
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)

In [3]:
flattened_lattice = avail_lattice_base.flatten()

## Seperate Lattice into Buildings 

In [4]:
s = tg.create_stencil("von_neumann", 1, 1)

In [5]:
env_lat_flat = avail_lattice_base.flatten()

adj_matrix = np.zeros((env_lat_flat.size,env_lat_flat.size), dtype=int)

all_vox_neighs_inds = avail_lattice_base.find_neighbours(s)
all_vox_neighs = all_vox_neighs_inds[:, 1:]
all_vox_inds = np.indices(all_vox_neighs.shape)[0]

adj_matrix[all_vox_inds.flatten(), all_vox_neighs.flatten()] = 1

adj_matrix[env_lat_flat==0, :] *= 0
adj_matrix[:, env_lat_flat==0] *= 0

g = nx.from_numpy_array(adj_matrix)

In [6]:
g.remove_nodes_from(list(nx.isolates(g)))

In [7]:
g_cc = nx.algorithms.connected_components(g)

buildingid_lat_flat = env_lat_flat * 0 - 1

for i, nodes in enumerate(g_cc):
    buildingid_lat_flat[list(nodes)] = i

In [8]:
buildingid_lat = tg.to_lattice(buildingid_lat_flat.reshape(avail_lattice_base.shape), avail_lattice_base)
flat_id=buildingid_lat.flatten()

## Retrieving ID's of cells in the 4 buildings 

In [9]:
Building_4= np.argwhere(flat_id==2)
Building_3= np.argwhere(flat_id==1)
Building_2= np.argwhere(flat_id==3)
Building_1= np.argwhere(flat_id==0)

## Env Lattices

In [10]:
Private_sector_Owned = (pickle.load( open( "Privately Owned Housing.p", "rb" ) ))
Social_Sector_Rental = (pickle.load( open( "Social Sector Rental Housing.p", "rb" ) ))
Free_Sector_rental = (pickle.load( open( "Free Sector Rental Housing.p", "rb" ) )) 
Restaurants_and_Cafes = (pickle.load( open( "Restaurants and Cafe.p", "rb" ) )) 
Offices = (pickle.load( open( "Offices.p", "rb" ) )) 
Retail = (pickle.load( open( "Retail Stores.p", "rb" ) )) 
Parking = (pickle.load( open( "Parking.p", "rb" ) )) 

## Stencils

In [11]:
# creating neighborhood definition
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)
#print(stencil_von_neumann)

In [12]:
# creating neighborhood definition 
stencil_squareness = tg.create_stencil("moore", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness[0,:,:] = 0 
stencil_squareness[2,:,:] = 0
stencil_squareness.set_index([0,0,0], 0)
stencil_squareness_t = np.transpose(stencil_squareness) 
#print(stencil_squareness_t)

In [13]:
# creating neighborhood definition 
stencil_squareness_von = tg.create_stencil("von_neumann", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness_von[0,:,:] = 0 
stencil_squareness_von[2,:,:] = 0
stencil_squareness_von.set_index([0,0,0], 0)
stencil_squareness_von_t = np.transpose(stencil_squareness_von) 
#print(stencil_squareness_von)

In [14]:
stencil_cuboid = tg.create_stencil("moore", 1, 1)
stencil_cuboid.set_index([0,0,0], 0)
#print(stencil_cuboid)

In [15]:
Stencil_Find_neighbours_cuboid = tg.create_stencil("moore", 1, 1)

In [16]:
stencil_full_floor = tg.create_stencil("moore",int(10/2) )*0
stencil_full_floor[:,:,int(10/2)] = 1
stencil_full_floor.set_index([0,0,0], 0)
#print(stencil_full_floor)

## Deriving all possible agent Center points

In [17]:
available_index_list= list(np.argwhere(flattened_lattice==1).flatten())

In [18]:
len(available_index_list)

1359

In [19]:
def all_possible_origins(flat_lattice,number_of_points_needed):
    reshaped_lattice= flat_lattice.reshape(99,4,16)*1
    reshaped_lattice_aggregated_value =np.sum(reshaped_lattice,axis=2)
    array = reshaped_lattice_aggregated_value.flatten()
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    
    indices =list(range(6,flat_lattice.size,16))
    points_1D=[]
    for points in range(number_of_points_needed):
        index= np.where(ranks==points)
        points_1D.append(indices[int(index[0])] )  
    return points_1D


In [20]:
## The number of voxels which each agent has to occupy There are five agents :[Blue,Green,Yellow,Red,Violet]
## The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1] 
mass_size = np.count_nonzero((avail_lattice==1)) 
P_H = 415 # Private Housing
S_H= 301 # Social Housing
F_H=403 # Free sector Housing
R_C= 19 # Restaurant and Cafe 
R= 73 # Retail stores
O=93 # Offices
P=51 # Parking


In [21]:
#Divide the available lattice into grids to generate the necessary locations for origins??

In [22]:
def ranking_origins(point_id,value_lattice,origin_number):
    selected_neighbours= avail_lattice_base.find_neighbours(Stencil_Find_neighbours_cuboid)[point_id]
    neighbour_vals_for_agent= value_lattice.flatten()[selected_neighbours]
    neighbour_vals_aggregated_for_agent =np.sum(neighbour_vals_for_agent,axis=1)
    all_points = np.array(point_id)
    array = neighbour_vals_aggregated_for_agent
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    
    a_lattice = ranks.flatten()
    Co_ordinates = []
    for item in range(0,origin_number):
        Co_ordinates.append(all_points[np.argwhere(a_lattice==item)].flatten()[0])    
        
    return Co_ordinates

In [23]:
flat_parking = Parking.flatten()[available_index_list] 
Agent_7_origins= available_index_list[np.argmax(flat_parking[10:])]
Agent_7_origins

20

In [24]:
PS_A_origin_1 = ranking_origins(Building_1.flatten(),Private_sector_Owned,70)
PS_A_origin_2 = ranking_origins(Building_4.flatten(),Private_sector_Owned,70)
SS_A_origin_1 = ranking_origins(Building_2.flatten(),Social_Sector_Rental,70)
SS_A_origin_2 = ranking_origins(Building_4.flatten(),Social_Sector_Rental,70)
FS_A_origin_1 = ranking_origins(Building_1.flatten(),Free_Sector_rental,70)
FS_A_origin_2 = ranking_origins(Building_4.flatten(),Free_Sector_rental,70)
RC_A_origin_1 = ranking_origins(available_index_list,Restaurants_and_Cafes,70)
O_A_origin_1 = ranking_origins(available_index_list,Offices,70)
RE_A_origin_1 = ranking_origins(available_index_list,Retail,70)

In [25]:
PS_A_origin_1

[2135,
 2124,
 12,
 23,
 1948,
 1959,
 2136,
 2125,
 1772,
 1783,
 1596,
 1607,
 1244,
 1255,
 1431,
 1420,
 188,
 199,
 1068,
 1079,
 364,
 375,
 892,
 903,
 540,
 551,
 716,
 727,
 2137,
 2126,
 13,
 24,
 2138,
 2127,
 14,
 25,
 2132,
 2143,
 20,
 31,
 26,
 15,
 2139,
 2128,
 1949,
 1960,
 16,
 27,
 1773,
 1784,
 2140,
 2129,
 1597,
 1608,
 1432,
 1421,
 1245,
 1256,
 28,
 17,
 200,
 189,
 1069,
 1080,
 1961,
 1950,
 376,
 365,
 904,
 893]

In [26]:
# agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id

    # definition of random/argmax occupancy on a 2d squarish stencil 
    def random_occupy_squareness(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
     
      # definition of random/argmax occupancy on a 3d cubish stencil
    def random_occupy_cubish(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
      
        
    def random_occupy_cubish_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_cuboid, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
        
    def argmax_occupy_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

        
     # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_moore(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)

        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]               
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        #print(neighs_availibility)
        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 9 neighbourhood
        
        #print(neighbourhood_details)
        one = neighs_availibility[1] + neighs_availibility[2] 
        two = neighs_availibility[0] + neighs_availibility[2] 
        three = neighs_availibility[1] + neighs_availibility[4] 
        four = neighs_availibility[0] + neighs_availibility[6] 
        five = neighs_availibility[2] + neighs_availibility[7] 
        six = neighs_availibility[3] + neighs_availibility[6] 
        seven = neighs_availibility[5] + neighs_availibility[7] 
        eight = neighs_availibility[6] + neighs_availibility[4] 
        neighbourhood_details = [one,two,three,four,five,six,seven,eight]
        
        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == 3:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + one_neighbour_factor
            elif neighbourhood_condition == 4:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + two_neighbour_factor
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail]
        #print(all_neighs_value_mod)   
        

        neighs_value_flattened = env.value.flatten()
        for val_mod in all_neighs_value_mod:
            for neigh in neighs :
                neighs_value_flattened[neigh]=val_mod
        
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        
        #print(free_neighs_value)
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    
    
         # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_von_neumann(self, env):
        
         # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    def one_neighbour_occupy_squareness_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_availibility_full_lattice = env.availibility.flatten()
        free_neighs_full_lattice = np.argwhere(neighs_availibility_full_lattice==1).flatten()
 
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_squareness.append(neighs)

        neighs_flatten = np.array(env.neigh_squareness).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        #neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)


        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )

        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
   
        
        
    def one_neighbour_occupy_cubish_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_availibility_full_lattice = env.availibility.flatten()
        free_neighs_full_lattice = np.argwhere(neighs_availibility_full_lattice==1).flatten()
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_cubish.append(neighs)

        neighs_flatten = np.array(env.neigh_cubish).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
                
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        
        #neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)

       # print(neighs_unq_base_value) 
        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )       
            
        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
        #print(selected_neigh_1D_id)
        #print(free_neighs_full_lattice)

            

In [27]:
# Agent init class

def initialize_agents_random_origin (stencil,avail_lattice):
    #finding origin 
    agn_num = 1
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    select_id = np.random.choice(len(avail_index), agn_num)
    agn_origins = tuple(avail_index[select_id].flatten()) 

    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

def initialize_agents_fixed_origin (stencil,avail_lattice,origin):
    #finding origin 
    agn_origins = np.unravel_index(origin,avail_lattice.shape)
    select_id = origin
    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

In [28]:
# environment class
class environment():
    def __init__(self, lattices, agents,number_of_iterations,method_name):
        self.availibility = lattices["availibility"]
        self.value = lattices["enviornment"]
        self.agent_origin = self.availibility 
        self.agents = agents
        self.update_agents()
        self.number_of_iterations = number_of_iterations
        self.method_name = method_name
        self.neigh_cubish = []
        self.neigh_squareness = []
    def update_agents(self):
        # making previous position available
      #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        self.availibility[tuple(self.agents.origin)] = self.agents.id
        # adding agent to the new position 
        self.agent_origin[tuple(self.agents.origin)] = self.agents.id
    
    def random_occupy_squareness_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_squareness(self)
        # update the agent states in environment
        self.update_agents()
        
    def random_occupy_cubish_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish(self)
        # update the agent states in environment
        self.update_agents()
    
    def random_occupy_cubish_von_neumann_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
    
    def argmax_occupy_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.argmax_occupy_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_moore(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_moore(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
      
    def one_neighbour_occupy_cubish_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_cubish_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        

In [29]:
def calculate_cost_for_positions(Location_list,Enviornment_lattice,number_of_cells_to_be_occupied):
    All_viz_options = []
    for indexing, item in enumerate(Location_list):
        #print(indexing)
        Agent = agent(np.unravel_index(item,init_avail_lattice.shape),stencil_cuboid,9)

        occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

        env = {"availibility": occ_lattice_sim,"enviornment": Enviornment_lattice}


        env_1 = environment(env,Agent,number_of_cells_to_be_occupied,"one_neighbour_occupy_cubish_behaviour")

        env_availability_viz = []
        env_list =[env_1]
        number_steps = max(map(lambda e:e.number_of_iterations,env_list))    
        for a in range(number_steps):
            # print(env.availibility)
            #print(env.agent_origin)

            for e in env_list:
                if a < e.number_of_iterations :
                    #print(a)
                    #print(e.number_of_iterations)
                    if e.method_name == "one_neighbour_occupy_squareness_moore":
                        e.one_neighbour_occupy_squareness_moore()

                    elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                        e.one_neighbour_occupy_cubish_agents()

                    elif e.method_name == "random_occupy_squareness_agents" :
                        e.random_occupy_squareness_agents()

                    elif e.method_name == "random_occupy_cubish_agents" :
                        e.random_occupy_cubish_agents()  

                    elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                        e.random_occupy_cubish_von_neumann_agents()                           

                    elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                        e.one_neighbour_occupy_squareness_von_neumann()                


                    elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                        e.one_neighbour_occupy_squareness_behaviour()  

                    elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                        e.one_neighbour_occupy_cubish_behaviour()  

                    elif e.method_name == "argmax_occupy_von_neumann" :
                        e.argmax_occupy_von_neumann()  

        env_availability_viz.append(e.availibility)

        All_viz_options.append(env_availability_viz)

    all_viz_array= np.array(All_viz_options)
    all_viz_array_shape = all_viz_array.shape
    reshaped_all_viz= all_viz_array.reshape(all_viz_array_shape[0]*all_viz_array_shape[1],all_viz_array_shape[2],all_viz_array_shape[3],all_viz_array_shape[4])
    sum_of_values = []
    for lattice in reshaped_all_viz:
        indexes= np.argwhere(lattice.flatten()==9).flatten()
        values= env["enviornment"].flatten()[indexes]
        total = int(np.sum(values))
        sum_of_values.append(total)
    return sum_of_values

In [30]:
"""
Cost_for_PH = np.array(calculate_cost_for_positions(Selected_positions,Private_sector_Owned,20)) # Private Housing
Cost_for_SH = np.array(calculate_cost_for_positions(Selected_positions,Social_Sector_Rental,20)) # Social Housing
Cost_for_FH = np.array(calculate_cost_for_positions(Selected_positions,Free_Sector_rental,20)) # Free sector Housing
Cost_for_RC = np.array(calculate_cost_for_positions(Selected_positions,Restaurants_and_Cafes,20)) # Restaurant and Cafe 
Cost_for_R = np.array(calculate_cost_for_positions(Selected_positions,Offices,20)) # Retail stores
Cost_for_O = np.array(calculate_cost_for_positions(Selected_positions,Retail,20)) # Offices
Cost_for_P = np.array(calculate_cost_for_positions(Selected_positions,Parking,20)) # Parking
"""

'\nCost_for_PH = np.array(calculate_cost_for_positions(Selected_positions,Private_sector_Owned,20)) # Private Housing\nCost_for_SH = np.array(calculate_cost_for_positions(Selected_positions,Social_Sector_Rental,20)) # Social Housing\nCost_for_FH = np.array(calculate_cost_for_positions(Selected_positions,Free_Sector_rental,20)) # Free sector Housing\nCost_for_RC = np.array(calculate_cost_for_positions(Selected_positions,Restaurants_and_Cafes,20)) # Restaurant and Cafe \nCost_for_R = np.array(calculate_cost_for_positions(Selected_positions,Offices,20)) # Retail stores\nCost_for_O = np.array(calculate_cost_for_positions(Selected_positions,Retail,20)) # Offices\nCost_for_P = np.array(calculate_cost_for_positions(Selected_positions,Parking,20)) # Parking\n'

In [31]:
def agent_based_on_origin(Origin,index,stencil):
    return agent(np.unravel_index(Origin,init_avail_lattice.shape),stencil,index)

In [32]:
Agents_P = agent_based_on_origin(PS_A_origin_1[0],10,stencil_cuboid)
Agents_P_2 = agent_based_on_origin(PS_A_origin_2[0],10,stencil_cuboid)
Agents_PR = agent_based_on_origin (309,100,stencil_squareness_t)

env_P = {"availibility": avail_lattice,"enviornment": Private_sector_Owned}
env_PR = {"availibility": avail_lattice,"enviornment": Parking}

env_1 = environment(env_P,Agents_P,100,"one_neighbour_occupy_cubish_behaviour")
env_2 = environment(env_P,Agents_P_2,315,"one_neighbour_occupy_cubish_behaviour")
env_3 = environment(env_PR,Agents_PR,51,"one_neighbour_occupy_squareness_behaviour")

In [33]:
env_availability_viz = []
env_list = [env_1,env_2,env_3]
number_steps = max(map(lambda e:e.number_of_iterations,env_list))
for a in range(number_steps):
    # print(env.availibility)
    #print(env.agent_origin)
    #print(a)
    for e in env_list:
        if a < e.number_of_iterations :
            #print(a)
            #print(e.number_of_iterations)
            if e.method_name == "one_neighbour_occupy_squareness_moore":
                e.one_neighbour_occupy_squareness_moore()
                
            elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                e.one_neighbour_occupy_cubish_agents()
                
            elif e.method_name == "random_occupy_squareness_agents" :
                e.random_occupy_squareness_agents()
            
            elif e.method_name == "random_occupy_cubish_agents" :
                e.random_occupy_cubish_agents()  
                
            elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                e.random_occupy_cubish_von_neumann_agents()                           
                
            elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                e.one_neighbour_occupy_squareness_von_neumann()                
                
                
            elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                e.one_neighbour_occupy_squareness_behaviour()  
                
            elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                e.one_neighbour_occupy_cubish_behaviour()  
            
            elif e.method_name == "argmax_occupy_von_neumann" :
                e.argmax_occupy_von_neumann()  
                
            env_availability_viz.append(e.availibility-1)

In [34]:
p = pv.Plotter(notebook=True)

base_lattice = env_availability_viz[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = env_availability_viz[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 1500
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

[6 6 6]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(168.72367820989257, 336.72367820989257, 303.72367820989257),
 (-105.0, 63.0, 30.0),
 (0.0, 0.0, 1.0)]

In [35]:
def abm_simulation(Agent_origin,a_id,stencil,val_lattice,number_of_voxels):
    Agent_sim = agent_based_on_origin(Agent_origin,a_id,stencil)
    env_lat = {"availibility": avail_lattice,"enviornment": val_lattice}
    env_sim = environment(env_lat,Agent_sim,number_of_voxels,"one_neighbour_occupy_cubish_behaviour")
    env_availability_viz = []
    env_list = [env_sim]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        #print(a)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  

                env_availability_viz.append(e.availibility-1)
    return env_availability_viz

In [36]:
mask_FS = avail_lattice.flatten()[FS_A_origin_2]==1
available_origin_FS= np.array(FS_A_origin_2)[mask_FS]

In [37]:
Free_sector_simulation= abm_simulation(FS_A_origin_1[2],200,stencil_cuboid,Free_Sector_rental,133)
Free_sector_simulation_2= abm_simulation(available_origin_FS[2],200,stencil_cuboid,Free_Sector_rental,270)

In [38]:
p = pv.Plotter(notebook=True)

base_lattice = Free_sector_simulation_2[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = Free_sector_simulation_2[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 1500
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

[6 6 6]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(168.72367820989257, 336.72367820989257, 303.72367820989257),
 (-105.0, 63.0, 30.0),
 (0.0, 0.0, 1.0)]

In [39]:
mask_SS_1 = avail_lattice.flatten()[SS_A_origin_2]==1
available_origin_SS_1= np.array(SS_A_origin_2)[mask_SS_1]
mask_SS_2 = avail_lattice.flatten()[SS_A_origin_1]==1
available_origin_SS_2= np.array(SS_A_origin_1)[mask_SS_2]

In [40]:
Social_Rental_simulation_1= abm_simulation(available_origin_SS_1[1],300,stencil_cuboid,Social_Sector_Rental,210)
Social_Rental_simulation_2= abm_simulation(available_origin_SS_2[1],300,stencil_cuboid,Social_Sector_Rental,90)

In [41]:
p = pv.Plotter(notebook=True)

base_lattice = Social_Rental_simulation_2[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = Social_Rental_simulation_2[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 1500
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

[6 6 6]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(168.72367820989257, 336.72367820989257, 303.72367820989257),
 (-105.0, 63.0, 30.0),
 (0.0, 0.0, 1.0)]

In [42]:
mask_O = avail_lattice.flatten()[O_A_origin_1]==1
available_origin_O= np.array(O_A_origin_1)[mask_O]

In [43]:
Offices_simulation= abm_simulation(available_origin_O[0],400,stencil_cuboid,Offices,93)

In [44]:
p = pv.Plotter(notebook=True)

base_lattice = Offices_simulation[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = Offices_simulation[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 1500
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

[6 6 6]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(168.72367820989257, 336.72367820989257, 303.72367820989257),
 (-105.0, 63.0, 30.0),
 (0.0, 0.0, 1.0)]

In [45]:
mask_RE = avail_lattice.flatten()[RE_A_origin_1]==1
available_origin_RE= np.array(RE_A_origin_1)[mask_RE]

In [46]:
Retail_simulation= abm_simulation(available_origin_RE[0],500,stencil_cuboid,Retail,73)

In [47]:
p = pv.Plotter(notebook=True)

base_lattice = Retail_simulation[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = Retail_simulation[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 1500
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

[6 6 6]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(168.72367820989257, 336.72367820989257, 303.72367820989257),
 (-105.0, 63.0, 30.0),
 (0.0, 0.0, 1.0)]

In [48]:
mask_RC = avail_lattice.flatten()[RC_A_origin_1]==1
available_origin_RC= np.array(RC_A_origin_1)[mask_RC]

In [49]:
available_origin_RC

array([760, 815, 762], dtype=int64)

In [50]:
Restaurants_simulation= abm_simulation(available_origin_RC[0],600,stencil_cuboid,Restaurants_and_Cafes,14)

In [51]:
p = pv.Plotter(notebook=True)

base_lattice = Restaurants_simulation[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = Restaurants_simulation[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 15
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

[6 6 6]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(168.72367820989257, 336.72367820989257, 303.72367820989257),
 (-105.0, 63.0, 30.0),
 (0.0, 0.0, 1.0)]

In [52]:
"""
for i, lattice in enumerate(Restaurants_simulation):
    csv_path = os.path.relpath('csv/8_abm_f_'+ f'{i:03}' + '.csv')
    lattice.to_csv(csv_path)
"""

"\nfor i, lattice in enumerate(Restaurants_simulation):\n    csv_path = os.path.relpath('csv/8_abm_f_'+ f'{i:03}' + '.csv')\n    lattice.to_csv(csv_path)\n"

In [53]:
avail_lattice

lattice([[[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
          [  0, 200, 200, 200, 200, 200, 200, 200, 200, 200,   0],
          [  0, 200, 200, 200, 200, 200, 200, 200, 200, 200,   0],
          [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
          [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
          [  0, 500, 500, 500,   0,   0,   0,   0,   0,   0,   0],
          [  0, 500, 500, 500,   0,   0,   0,   0,   0,   0,   0],
          [  0, 500, 500, 500,   0,   0,   0,   0,   0,   0,   0],
          [  0, 500, 500, 500,   0,   0,   0,   0,   0,   0,   0],
          [  0, 500, 500, 500,   0,   0,   0,   0,   0,   0,   0],
          [  0, 500, 500, 500,   0,   0,   0,   0,   0,   0,   0],
          [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
          [  0, 100, 200, 200, 200, 200, 200, 200, 200, 200,   0],
          [  0, 100, 200, 200, 200, 200, 200, 200, 200, 200,   0],
          [  0, 100, 200, 200, 200, 200, 200, 200, 200, 200,  

In [54]:

#Zoning_mtrx = pickle.dump( avail_lattice, open( "Zoning output.p", "wb" ) )
#pickle_matrix = pickle.load( open( "dist_matrix_toy_problem.p", "rb" ) ) 

In [55]:
type(avail_lattice)

topogenesis.datastructures.datastructures.lattice

In [56]:
avail_lattice.centroids 

cloud([[-210.,   24.,    6.],
       [-210.,   24.,   12.],
       [-210.,   24.,   18.],
       [-210.,   24.,   24.],
       [-210.,   24.,   30.],
       [-210.,   24.,   36.],
       [-210.,   24.,   42.],
       [-210.,   24.,   48.],
       [-210.,   24.,   54.],
       [-210.,   30.,    6.],
       [-210.,   30.,   12.],
       [-210.,   30.,   18.],
       [-210.,   30.,   24.],
       [-210.,   30.,   30.],
       [-210.,   30.,   36.],
       [-210.,   30.,   42.],
       [-210.,   30.,   48.],
       [-210.,   30.,   54.],
       [-210.,   48.,    6.],
       [-210.,   48.,   12.],
       [-210.,   48.,   18.],
       [-210.,   54.,    6.],
       [-210.,   54.,   12.],
       [-210.,   54.,   18.],
       [-210.,   60.,    6.],
       [-210.,   60.,   12.],
       [-210.,   60.,   18.],
       [-210.,   66.,    6.],
       [-210.,   66.,   12.],
       [-210.,   66.,   18.],
       [-210.,   72.,    6.],
       [-210.,   72.,   12.],
       [-210.,   72.,   18.],
       [-2

In [57]:
#csv_path = os.path.relpath('Zoning_envelop.csv')
#avail_lattice.to_csv(csv_path)

In [58]:
P_index = np.argwhere(avail_lattice.flatten()==10).flatten()
S_index = np.argwhere(avail_lattice.flatten()==300).flatten()
FS_index = np.argwhere(avail_lattice.flatten()==200).flatten()
R_index = np.argwhere(avail_lattice.flatten()==600).flatten()
O_index = np.argwhere(avail_lattice.flatten()==400).flatten()
RE_index = np.argwhere(avail_lattice.flatten()==500).flatten()
PA_index = np.argwhere(avail_lattice.flatten()==100).flatten()

In [59]:
PC= np.sum(Private_sector_Owned.flatten()[P_index])
SC= np.sum(Social_Sector_Rental.flatten()[S_index])
FSC= np.sum(Free_Sector_rental.flatten()[FS_index])
RC= np.sum(Restaurants_and_Cafes.flatten()[R_index])
OC= np.sum(Offices.flatten()[O_index])
REC= np.sum(Retail.flatten()[RE_index])
PAC= np.sum(Parking.flatten()[PA_index])
total_cost = PC + SC + FSC+ RC +  OC + REC + PAC

In [60]:
total_cost

lattice(8236.13982676)

In [61]:
PAC

lattice(462.22222222)

In [62]:
Zoning_output = pickle.dump( avail_lattice, open( "Zoning_output.p", "wb" ) )

In [63]:
csv_path = os.path.relpath('Zoning_output.csv')
avail_lattice.to_csv(csv_path)